In [ ]:
#Загрузим необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
import os
import zipfile

import cv2
from imageio import imread

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.compiler.tensorrt import trt_convert as trt

from tensorflow.keras.applications.imagenet_utils import preprocess_input

from livelossplot.tf_keras import PlotLossesCallback

In [ ]:
#Настройка пути с датасетом
dir_path = '.'

#Здесь должны быть фото по папкам
base_dir = Path(dir_path + '/train')

In [ ]:
#Зададим размер batch и размер изображения
BATCH_SIZE = 64
IMG_SHAPE  = 224

#Создадим необходимые train (80%) и validation (20%) datasets
#Зададим некоторые параметры аугментации

data_image_gen = ImageDataGenerator(
                                    preprocessing_function=preprocess_input,
#                                     rescale=1./255,
                                    rotation_range=40, #Добавить поворот
                                    shear_range=0.2, #Добавить сдвиг
                                    zoom_range=0.2, #Добавить увеличение
                                    horizontal_flip=True, #Добавить зеркальный поворот
                                    fill_mode="nearest", #Заполняем пробелы
                                    validation_split=0.2)

train_data = data_image_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                                directory=base_dir,
                                                shuffle=True,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                class_mode='categorical',
                                                subset='training')

valid_data = data_image_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                                directory=base_dir,
                                                shuffle=True,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                class_mode='categorical',
                                                subset='validation')

In [ ]:
#Изображения и лэйблы
train_images, train_labels = next(train_data)
val_images, val_labels = next(valid_data)

#Количество людей для распознавания
n_faces = train_labels.shape[1]

In [ ]:
#Модель
model_VGGFace = tf.keras.Sequential([tf.keras.layers.ZeroPadding2D((1,1),input_shape=(224,224, 3)),
                                           tf.keras.layers.Convolution2D(64, (3, 3), activation='relu'),
                                           tf.keras.layers.ZeroPadding2D((1,1)),
                                           tf.keras.layers.Convolution2D(64, (3, 3), activation='relu'),
                                           tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)),

                                           tf.keras.layers.ZeroPadding2D((1,1)),
                                           tf.keras.layers.Convolution2D(128, (3, 3), activation='relu'),
                                           tf.keras.layers.ZeroPadding2D((1,1)),
                                           tf.keras.layers.Convolution2D(128, (3, 3), activation='relu'),
                                           tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)),

                                           tf.keras.layers.ZeroPadding2D((1,1)),
                                           tf.keras.layers.Convolution2D(256, (3, 3), activation='relu'),
                                           tf.keras.layers.ZeroPadding2D((1,1)),
                                           tf.keras.layers.Convolution2D(256, (3, 3), activation='relu'),
                                           tf.keras.layers.ZeroPadding2D((1,1)),
                                           tf.keras.layers.Convolution2D(256, (3, 3), activation='relu'),
                                           tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)),

                                           tf.keras.layers.ZeroPadding2D((1,1)),
                                           tf.keras.layers.Convolution2D(512, (3, 3), activation='relu'),
                                           tf.keras.layers.ZeroPadding2D((1,1)),
                                           tf.keras.layers.Convolution2D(512, (3, 3), activation='relu'),
                                           tf.keras.layers.ZeroPadding2D((1,1)),
                                           tf.keras.layers.Convolution2D(512, (3, 3), activation='relu'),
                                           tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)),


                                           tf.keras.layers.ZeroPadding2D((1,1)),
                                           tf.keras.layers.Convolution2D(512, (3, 3), activation='relu'),
                                           tf.keras.layers.ZeroPadding2D((1,1)),
                                           tf.keras.layers.Convolution2D(512, (3, 3), activation='relu'),
                                           tf.keras.layers.ZeroPadding2D((1,1)),
                                           tf.keras.layers.Convolution2D(512, (3, 3), activation='relu'),
                                           tf.keras.layers.MaxPooling2D((2,2), strides=(2,2)),

                                           tf.keras.layers.Convolution2D(4096, (7, 7), activation='relu'),
                                           tf.keras.layers.Dropout(0.5),
                                           tf.keras.layers.Convolution2D(4096, (1, 1), activation='relu'),
                                           tf.keras.layers.Dropout(0.5),
                                           tf.keras.layers.Convolution2D(2622, (1, 1)),

                                           tf.keras.layers.Flatten()])

#загрузим веса VGGFace
model_VGGFace.load_weights(dir_path + '/vgg_face_weights.h5')

#Посмотрим на структуру  модели
model_VGGFace.summary()

In [ ]:
#Возьмём только основную часть модели до слоя

model_base_VGGFace = tf.keras.Model([model_VGGFace.input],
                                    model_VGGFace.get_layer('flatten').output)

#Заморозим базовую модель (так веса модели сформированы датасетом VGGFace)
#Если веса imagenet, то все слои разморозить (=True)
model_base_VGGFace.trainable = True

In [ ]:
#Допишем модель
faces_model_base_VGGFace = tf.keras.Sequential([model_base_VGGFace,
                                                  tf.keras.layers.Dense(512, activation='relu'),
                                                  tf.keras.layers.Dense(n_faces, activation='softmax')])

In [ ]:
#Параметры модели
optim = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = tf.keras.losses.CategoricalCrossentropy()
metric = tf.keras.metrics.categorical_accuracy

#Компиляция
faces_model_base_VGGFace.compile(optimizer=optim, loss=loss, metrics=metric)

In [ ]:
# #Сохраним наилучший результат модели
# model_best_path = Path(dir_path + '/model_best_VGGFace')
# model_best_path.mkdir(exist_ok=True)
# model_best_filename = 'checkpoint_best.h5'
# model_best_path_total = str(model_best_path/model_best_filename)
# checkpoint = tf.keras.callbacks.ModelCheckpoint(model_best_path_total,
#                                                 monitor='val_accuracy',
#                                                 verbose=1,
#                                                 save_best_only=True,
#                                                 mode='max')

In [ ]:
#Обучение

EPOCHS = 5
history = faces_model_base_VGGFace.fit(train_data,
                                         validation_data=valid_data,
                                         epochs=EPOCHS,
                                         callbacks=[PlotLossesCallback()])

In [ ]:
#Сохраним модель
faces_model_base_VGGFace.save(dir_path + '/checkpoint_best_2.h5')